In [ ]:
# ==========================================
# GRAD-CAM FOR SATELLITE IMAGERY (PyTorch)
# ==========================================

import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

import matplotlib.pyplot as plt
import cv2


# ==========================================
# DEVICE & PATHS
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = Path(r"C:\Users\vansh\Desktop\satellite imagery\project_root\images")
CSV_PATH = Path(r"C:\Users\vansh\Desktop\satellite imagery\project_root\final_multimodal.csv")


# ==========================================
# LOAD DATA (ONLY FOR SAMPLE SELECTION)
# ==========================================
fusion_df = pd.read_csv(CSV_PATH)
fusion_df["id"] = fusion_df["id"].astype(int)

# Keep only rows that have downloaded images
image_ids = sorted([
    int(float(p.stem.split(".")[0]))
    for p in IMAGE_DIR.glob("*.png")
])

fusion_df = (
    fusion_df[fusion_df["id"].isin(image_ids)]
    .drop_duplicates(subset="id")
    .reset_index(drop=True)
)

print("Usable samples:", fusion_df.shape)


# ==========================================
# SELECT LOW & HIGH PRICED SAMPLES
# ==========================================
cheap_samples = fusion_df.nsmallest(3, "price")
expensive_samples = fusion_df.nlargest(3, "price")


# ==========================================
# IMAGE PREPROCESSING (SAME AS TRAINING)
# ==========================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# ==========================================
# LOAD PRETRAINED RESNET18 (FEATURE EXTRACTOR)
# ==========================================
model = models.resnet18(
    weights=models.ResNet18_Weights.IMAGENET1K_V1
)

# Remove classifier (we only need features)
model.fc = nn.Identity()

model.to(device)
model.eval()

for p in model.parameters():
    p.requires_grad = False


# ==========================================
# GRAD-CAM IMPLEMENTATION (FIXED)
# ==========================================
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None

        # Register hooks
        target_layer.register_forward_hook(self._forward_hook)
        target_layer.register_full_backward_hook(self._backward_hook)

    def _forward_hook(self, module, input, output):
        self.activations = output.detach()

    def _backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def generate(self, input_tensor):
        self.model.zero_grad()

        output = self.model(input_tensor)
        output.mean().backward()

        # Global average pooling of gradients
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1)

        cam = torch.relu(cam)
        cam -= cam.min()
        cam /= (cam.max() + 1e-8)

        return cam.squeeze().cpu().numpy()


# ==========================================
# IMAGE LOADER
# ==========================================
def load_image(property_id):
    path_1 = IMAGE_DIR / f"{property_id}.png"
    path_2 = IMAGE_DIR / f"{property_id}.0.png"

    img_path = path_1 if path_1.exists() else path_2
    img = Image.open(img_path).convert("RGB")

    return img, img_path


# ==========================================
# VISUALIZATION FUNCTION
# ==========================================
def show_gradcam(property_id, price, title):
    img, img_path = load_image(property_id)

    input_tensor = transform(img).unsqueeze(0).to(device)
    input_tensor.requires_grad = True

    cam = gradcam.generate(input_tensor)

    # Resize CAM
    cam = cv2.resize(cam, (224, 224))
    heatmap = cv2.applyColorMap(
        np.uint8(255 * cam),
        cv2.COLORMAP_JET
    )
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    img_np = np.array(img.resize((224, 224)))
    overlay = 0.6 * img_np + 0.4 * heatmap
    overlay = overlay.astype(np.uint8)

    plt.figure(figsize=(4, 4))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title(f"{title}\n₹{price:,}")
    plt.show()


# ==========================================
# INITIALIZE GRAD-CAM
# ==========================================
gradcam = GradCAM(
    model=model,
    target_layer=model.layer4
)


# ==========================================
# VISUALIZE LOW-PRICED PROPERTIES
# ==========================================
for _, row in cheap_samples.iterrows():
    show_gradcam(
        property_id=row["id"],
        price=row["price"],
        title="Low-Priced Property"
    )


# ==========================================
# VISUALIZE HIGH-PRICED PROPERTIES
# ==========================================
for _, row in expensive_samples.iterrows():
    show_gradcam(
        property_id=row["id"],
        price=row["price"],
        title="High-Priced Property"
    )


In [8]:
!pip install opencv-python

